I’ve got the first set of services created for the bike lock and trail monitor.  These are CRUD services only, but I think that’s really all we need for now.  They are published here: http://gwsportal.azurewebsites.net/api
 
This is the flow that I think the UI should call/use these APIs:
 
1.       User approaches the kiosk and scans their badge (api/userapi?id=<rfid>) Get the user information and let the continue if they are ACTIVE.

2.       Get a list of bikes (api/bikesapi).  Use this list to display the bikes and their current status.

3.       User selects a bike or bikes to use (api/bikesapi?id<Id>) This will be  PUT operation on each bike selected to update its status to OUT.

4.       Log the transaction (api/bikelogsapi) This will be a POST operation to insert a new log record into the database.

5.       User returns the bike(s) (api/bikesapi?id<Id>) This will be  PUT to update the status to IN.

6.       Log the transaction (api/bikelogsapi) This will be a POST operation to insert a new log record into the database.

 
I will type up a full API reference later and include the trail monitor stuff, but the different APIs for the bike lock are broken out as follows.  Keep in mind any PUT,POST, or DELETE operation will need authentication.  I still need to create that and will email as soon as it’s done.
 
UsersApi
-          Object:

o   Id (This is the database ID)

o   RFId (This is the badge id assigned to the user)

o   First name

o   Last name

o   Status (ACTIVE or INACTIVE)

o   Self (this is a url that points back to the object.  Really not needed in what we’re doing)

-          Get all:  http://gwsportal.azurewebsites.net/usersapi

o   This method returns a list (array) of Json objects of type User

-          Get single: http://gwsportal.azurewebsites.net/usersapi?id=<rfid>

o   This method returns a single Json object of type User

-          Put: http://gwsportal.azurewebsites.net/usersapi?id=<Id>&&user=<Json object of type User>

o   This method is used to update a user record in the database.  You’ll pass in the database ID and a Json object of type user

-          Post: http://gwsportal.azurewebsites.net/usersapi?user=<Json object of type User>

o   This method is used to insert a new user record in the database.  Pass in a Json object of type user.

 
BikesApi
-          Object:

o   Id (This is the database ID)

o   BikeName

o   SerialNo

o   Status (IN or OUT)

o   Self (this is a url that points back to the object.  Really not needed in what we’re doing)

-          Get all:  http://gwsportal.azurewebsites.net/bikesapi

o   This method returns a list (array) of Json objects of type Bike

-          Get single: http://gwsportal.azurewebsites.net/bikesapi?id=<Id>

o   This method returns a single Json object of type Bike

-          Put: http://gwsportal.azurewebsites.net/bikesapi?id=<Id>&&user=<Json object of type Bike>

o   This method is used to update a bike record in the database.  You’ll pass in the database ID and a Json object of type Bike

-          Post: http://gwsportal.azurewebsites.net/bikesapi?user=<Json object of type Bike>

o   This method is used to insert a new bike record in the database.  Pass in a Json object of type Bike.

 
BikeLogsApi
-          Object:

o   Id (This is the database ID)

o   UserId (This is the RFID assigned to the user)

o   BikeId (This is the bikes database ID)

o   TransactionDate (Datetime stamp that the bike is checked in or out)

o   TransactionType (IN or OUT)

o   Self (this is a url that points back to the object.  Really not needed in what we’re doing)

-          Get all:  http://gwsportal.azurewebsites.net/bikelogsapi

o   This method returns a list (array) of Json objects of type BikeLog

-          Get single: http://gwsportal.azurewebsites.net/bikelogsapi?id=<Id>

o   This method returns a single Json object of type BikeLog

-          Put: http://gwsportal.azurewebsites.net/bikelogsapi?id=<Id>&&user=<Json object of type BikeLog>

o   This method is used to update a log record in the database.  You’ll pass in the database ID and a Json object of type BikeLog

-          Post: http://gwsportal.azurewebsites.net/bikelogsapi?user=<Json object of type BikeLog>

o   This method is used to insert a new log record in the database.  Pass in a Json object of type BikeLog.



In [28]:
import requests

In [2]:
list_of_users = requests.get('http://gwsportal.azurewebsites.net/api/usersapi/')

In [4]:
print(list_of_users)
list_of_users.json()

<Response [200]>


[{'FirstName': 'Eric',
  'Id': 1,
  'LastName': 'Cartman',
  'RFId': 12345,
  'Self': 'api/users/1',
  'Status': 'ACTIVE'},
 {'FirstName': 'Stan',
  'Id': 2,
  'LastName': 'Marsh',
  'RFId': 12346,
  'Self': 'api/users/2',
  'Status': 'ACTIVE'},
 {'FirstName': 'Kyle',
  'Id': 3,
  'LastName': 'BRoflovski',
  'RFId': 12347,
  'Self': 'api/users/3',
  'Status': 'ACTIVE'},
 {'FirstName': 'Kenny',
  'Id': 4,
  'LastName': 'McCormick',
  'RFId': 12348,
  'Self': 'api/users/4',
  'Status': 'ACTIVE'},
 {'FirstName': 'Butters',
  'Id': 5,
  'LastName': 'Stoch',
  'RFId': 12349,
  'Self': 'api/users/5',
  'Status': 'ACTIVE'},
 {'FirstName': 'Pip',
  'Id': 6,
  'LastName': 'Pirrup',
  'RFId': 12350,
  'Self': 'api/users/6',
  'Status': 'INACTIVE'},
 {'FirstName': 'Alex',
  'Id': 7,
  'LastName': 'Glick',
  'RFId': 12351,
  'Self': 'api/users/7',
  'Status': 'INACTIVE'},
 {'FirstName': 'Chris',
  'Id': 8,
  'LastName': 'Donnely',
  'RFId': 12352,
  'Self': 'api/users/8',
  'Status': 'INACTIVE'}]

In [40]:
ind_user = requests.get('http://gwsportal.azurewebsites.net/api/usersapi?id=12351')

In [41]:
ind_user.json()['FirstName']

'Alex'

In [25]:
bikes = requests.get('http://gwsportal.azurewebsites.net/api/bikelogsapi')

In [26]:
bikes

<Response [200]>

In [27]:
bikes.json()

[]

In [20]:
from tkinter import *
import time
import subprocess
import re

In [ ]:
class Application(Frame):
    def bike(self, bike):
        print("This is Bike", bike)
        self.checkout = Button(self, text = "Checkout", command = lambda: self.checkoutbike(bike))
        self.checkout.pack()
        self.bike1.destroy()
        self.bike2.destroy()
    
    def checkoutbike(self, bike):
        allowed = False
        print("Checking out bike", bike)
        self.entrythingy = Entry()
        self.entrythingy.pack()
        self.checkout.destroy()
        # here is the application variable
        self.contents = StringVar()
        # set it to some value
        self.contents.set("Scan your RFID")
        # tell the entry widget to watch this variable
        self.entrythingy["textvariable"] = self.contents

        p = subprocess.Popen('./test.sh', stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        time.sleep(10)
        output = str(p.stdout.readline())
        code = re.findall('\d+', output)[0]
        auth = requests.get('http://gwsportal.azurewebsites.net/api/usersapi?id='+code)
        authjson = auth.json()
        if 'FirstName' in authjson:
            allowed = True
            print("Authenticated")
        else:
            print("Not Authenticated")
            allowed = False
        if allowed:
            # Opens lock
            # Puts new bike log
            pass
    
    def createWidgets(self):
        self.bike1 = Button(self, text = "Bike 1", command = lambda: self.bike(1))

        self.bike1.pack({"side": "left"})
        
        self.bike2 = Button(self, text = "Bike 2", command = lambda: self.bike(2))

        self.bike2.pack({"side": "left"})


    def __init__(self, master=None):
        Frame.__init__(self, master)
        self.pack()
        self.createWidgets()

root = Tk()
app = Application(master=root)
app.mainloop()
root.destroy()

This is Bike 2
Checking out bike 2
Authenticated

In [ ]:
print(auth)

In [53]:
code = "12351"
auth = requests.get('http://gwsportal.azurewebsites.net/api/usersapi?id='+code)
if 'FirstName' in auth.json():
    print(auth.json()['FirstName'])
    allowed = True
    print("Authenticated")
else:
    print("Not Authenticated")
    allowed = False
if allowed:
    # Opens lock
    # Puts new bike log
    pass

Alex
Authenticated
